In [ ]:
import gzip
import tqdm

In [ ]:
major_minor_dict = {chrom: {} for chrom in snakemake.params['chrom']}
maf_prefix = snakemake.params["maf_prefix"]

for chrom in snakemake.params["chrom"]:
    glob_maf_path = f"{maf_prefix}/allSamples/{chrom}/{chrom}_allSamples_snps.mafs.gz"
    glob_mafs = gzip.open(glob_maf_path,'rb').readlines()

    for i, l in enumerate(glob_mafs):
        if i != 0:
            sl = l.strip().split(b"\t")
            pos = sl[1].decode("utf-8")
            REF = sl[2]
            ALT = sl[3]
            major_minor_dict[chrom][pos] = [REF, ALT]

In [ ]:
def extract_positions(city, habitat):
    pos_index_dict = {chrom: {} for chrom in snakemake.params['chrom']}
    for chrom in snakemake.params["chrom"]:
        pos_path = f"{maf_prefix}/byCity/{city}/{chrom}/{city}_{habitat}_{chrom}_snps.pos.gz"
        with gzip.open(pos_path,'rb') as pos:
            lines = pos.readlines() 
            for i, l in enumerate(lines):
                if i != 0:
                    sl = l.strip().split(b"\t")
                    pos = sl[1].decode('utf-8')
                    pos_index_dict[chrom][pos] = i
    return pos_index_dict

city_pos_index_dict = {city: {hab: [] for hab in snakemake.params["habitats"]} for city in snakemake.params["cities"]}
for city in tqdm.tqdm(snakemake.params["cities"]):
    for hab in snakemake.params["habitats"]:
        city_pos_index_dict[city][hab] = extract_positions(city, hab)

In [ ]:
missing_dict = {city: {hab: {chrom: [] for chrom in snakemake.params["chrom"]} for hab in snakemake.params["habitats"]} for city in snakemake.params["cities"]}

def map_global_site_to_city_pos_indices(city, hab):
    index_mapping_dict = {chrom: {} for chrom in snakemake.params['chrom']}
    for chrom in snakemake.params["chrom"]:
        for g_pos in major_minor_dict[chrom].keys():
            try:
                pos_idx = city_pos_index_dict[city][hab][chrom][g_pos]
                index_mapping_dict[chrom][g_pos] = pos_idx
            except KeyError:
                # print(f"{chrom}: {gp} missing from {hab} habitat in {city}")
                missing_dict[city][hab][chrom].append(g_pos)
    return index_mapping_dict

city_index_mapping_dict = {city: {hab: [] for hab in snakemake.params["habitats"]} for city in snakemake.params["cities"]}
for city in tqdm.tqdm(snakemake.params["cities"]):
    for hab in snakemake.params["habitats"]:
        city_index_mapping_dict[city][hab] = map_global_site_to_city_pos_indices(city, hab)

In [ ]:
combined_missing_site_dict = {chrom: set() for chrom in snakemake.params['chrom']}

for city in tqdm.tqdm(snakemake.params["cities"]):
    for hab in snakemake.params["habitats"]:
        for chrom in snakemake.params["chrom"]:
            combined_missing_site_dict[chrom].update(missing_dict[city][hab][chrom])

In [ ]:
nucl_index_dict = {b'A': 0, b'C': 1, b'G': 2, b'T': 3}
out_prefix = snakemake.params["out_prefix"]

def write_city_geno_file(city):
    outpath = f"{out_prefix}/{city}"

    if not os.path.exists(outpath):
        os.makedirs(outpath)

    outfile = f"{outpath}/{city}.geno"
    with open(outfile, "w") as fout:
        for chrom in snakemake.params["chrom"]:
            urb_counts_path = f"{maf_prefix}/byCity/{city}/{chrom}/{city}_urban_{chrom}_snps.counts.gz"
            rur_counts_path = f"{maf_prefix}/byCity/{city}/{chrom}/{city}_rural_{chrom}_snps.counts.gz"
            # urb_pos_path = maf_prefix + f"byCity/{city}/{chrom}/{city}_{hab}_{chrom}_snps.pos.gz"
            # rur_pos_path = maf_prefix + f"byCity/{city}/{chrom}/{city}_{hab}_{chrom}_snps.pos.gz"
            
            urban_counts = gzip.open(urb_counts_path,'rb').readlines()
            rural_counts = gzip.open(rur_counts_path,'rb').readlines()
            # urban_pos = gzip.open(urb_pos_path,'rb').readlines()
            # rural_pos = gzip.open(rur_pos_path,'rb').readlines()
            
            for g_pos in major_minor_dict[chrom].keys():
                if g_pos in combined_missing_site_dict[chrom]:
                    pass
                else:
                    try:
                        urban_idx = city_pos_index_dict[city]["urban"][chrom].get(g_pos, None)
                        rural_idx = city_pos_index_dict[city]["rural"][chrom].get(g_pos, None)
                        
                        # print(urban_counts[urban_idx].strip().split(b"\t"), rural_counts[rural_idx].strip().split(b"\t"))
                        # print(REF, ALT)
                        # assert g_pos == urban_pos[urban_idx].strip().split(b"\t")[1]
                        # assert g_pos == rural_pos[urban_idx].strip().split(b"\t")[1]
                        REF = major_minor_dict[chrom][g_pos][0]
                        ALT = major_minor_dict[chrom][g_pos][1]
                        
                        urban_ref = urban_counts[urban_idx].strip().split(b"\t")[nucl_index_dict[REF]].decode('utf-8')
                        urban_alt = urban_counts[urban_idx].strip().split(b"\t")[nucl_index_dict[ALT]].decode('utf-8')
                        rural_ref = rural_counts[rural_idx].strip().split(b"\t")[nucl_index_dict[REF]].decode('utf-8')
                        rural_alt = rural_counts[rural_idx].strip().split(b"\t")[nucl_index_dict[ALT]].decode('utf-8')
                        # print(urban_ref, urban_alt, rural_ref, rural_alt)
                        # print("========================")
                        fout.write(f"{urban_ref} {urban_alt} {rural_ref} {rural_alt}\n")
                    except IndexError:
                        print(f"{chrom}: {g_pos}")
                        break



In [ ]:
for city in tqdm.tqdm(snakemake.params["cities"]):
    write_city_geno_file(city)

In [ ]:
for cont_file in snakemake.output["perCity_cont"]:
    with open(cont_file, "w") as fout:
        fout.write("1 -1")

In [ ]:
for city in tqdm.tqdm(snakemake.params["cities"]):
    with open(f"{out_prefix}/{city}/{city}.poolsize", "w") as fout:
        for hab in snakemake.params["habitats"]:
            bam_list_file = [f for f in snakemake.input["perCity_bams"] if city in f and hab in f][0]
            with open(bam_list_file, "r") as fin:
                lines = fin.readlines()
                if hab == "urban":
                    num_urban_alleles = len(lines) * 2
                else:
                    num_rural_alleles = len(lines) * 2
        fout.write(f"{num_urban_alleles} {num_rural_alleles}")

In [ ]:
with open(snakemake.output["site_order"], "w") as fout:
    for chrom in snakemake.params["chrom"]:
        for gp in major_minor_dict[chrom].keys():
            if gp in combined_missing_site_dict[chrom]:
                pass
            else:
                fout.write(f"{chrom}\t{gp}\n")

In [ ]:
with open(snakemake.output["miss"], "w") as fout:
    for city in tqdm.tqdm(snakemake.params["cities"]):
        for hab in snakemake.params["habitats"]:
            for chrom in snakemake.params["chrom"]:
                for pos in missing_dict[city][hab][chrom]:
                    # print(f"{chrom}: {pos} missing from {hab} habitat in {city}")
                    fout.write(f"{city}\t{hab}\t{chrom}\t{pos}\n")